In [7]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from datasets import load_from_disk
import pandas as pd
from tqdm.notebook import tqdm
import time

load_dotenv()

True

In [8]:
ground_truth_translations = pd.read_csv("../../data/english_to_urdu_translations/context_translations_trimmed.csv")

In [9]:
display(ground_truth_translations.head())

,context_eng,context_urdu
0,"Existing rules, etc., to continue\n\nUntil the...",۴۲۲۔ ۲۲۔ جب تک کہ متعلقہ مقننہ آ...
1,Protection against retrospective punishment\n\...,کوئی قانون کسی شخص کو مؤثر بہ ماضی سزا سے تحفظ...
2,"Oath of Office\n\nBefore entering upon office,...",ﻋﮩﺪﮮ ﮐﺎ ﺣﻠﻒ\n\nﻋﮩﺪﮮ ﭘﺮ ﻓﺎﺋﺰ ﮨﻮﻧﮯ ﺳﮯ ﻗﺒﻞ، ﮔﻮﺭﻧﺮ...
3,Oath of President\n\nBefore entering upon offi...,صدر کا حلف\n\nعہدہ سنبھالنے سے قبل، صدر پاکستا...
4,National language\n\n(1) The National language...,قومی زبان\n\n(1) پاکستان کی قومی زبان اردو ہے ...


In [15]:
for i in range(len(ground_truth_translations)):
    print(ground_truth_translations.iloc[i].context_eng)
    break

Existing rules, etc., to continue

Until the appropriate Legislature makes a law under Article 240, all rules and orders in force immediately before the commencing day shall, so far as consistent with the provisions of the Constitution, continue in force and may be amended from time to time by the Federal Government or, as the case may be the Provincial Government.


In [10]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [16]:
def get_urdu_response(context_eng, model_name="gemini-1.5-pro-002"):
    # Create the model
    generation_config = {
        "temperature": 0.0,
        "max_output_tokens": 1024,
        "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
        model_name=model_name,
        generation_config=generation_config,
        system_instruction="You are an expert translator specializing in English to Urdu legal translations."
    )

    chat_session = model.start_chat(history=[])

    prompt = f"""Translate the following legal text from English to Urdu. Provide only the translated text in Urdu script, without any additional commentary or explanation.

English Text:
```{context_eng}```

Urdu Translation:"""

    response = chat_session.send_message(prompt)

    return response.text.strip()

In [20]:
def get_and_save_answers(model_name):
    predicted_answers = []
    max_tries = 5  # Move max_tries outside loop

    for i in tqdm(range(len(ground_truth_translations)), desc="Processing responses", unit="item"):
        context = ground_truth_translations.iloc[i].context_eng

        tries = 0
        predicted_answer = None

        # Retry mechanism with max_tries
        while tries < max_tries:
            tries += 1
            try:
                predicted_answer = get_urdu_response(context_eng=context, model_name=model_name)
                break  # If successful, exit loop
            except Exception as error:
                error_msg = f"Index: {i} - ERROR OCCURRED: {error}"
                predicted_answer = error_msg

                time.sleep(15)  # Wait before retrying

        if tries >= max_tries:
            print(f"Max retries exceeded for index {i}: {predicted_answer}")

        predicted_answers.append(predicted_answer)

    # Print first 5 answers (avoid index error)
    for i in range(min(5, len(predicted_answers))):
        print(predicted_answers[i])

    # Add predictions to the DataFrame and save
    df2 = ground_truth_translations.copy()
    df2[f'predicted_translation_{model_name}'] = predicted_answers

    display(df2.head())  # Assuming display works in your environment

    # Save to CSV
    df2.to_csv(f'../../data/model_translation_responses/context_translations_{model_name}.csv', index=False)

In [21]:
models = [
    'gemini-1.5-pro-002',
    'gemini-1.5-flash-002'
]

In [22]:
for model in models:
    print(f'Model: {model}')

    get_and_save_answers(model)

    print('---' * 10, 'Done', '---' * 10)

Model: gemini-1.5-pro-002


Processing responses:   0%|          | 0/50 [00:00<?, ?item/s]

موجودہ قوانین وغیرہ کا جاری رہنا۔

آئین کے آرٹیکل 240 کے تحت مناسب قانون ساز ادارے کی جانب سے قانون سازی تک، آغاز کے دن سے فوری طور پر قبل نافذ العمل تمام قوانین اور احکامات، جہاں تک آئین کی دفعات سے مطابقت رکھتے ہیں، نافذ العمل رہیں گے اور وفاقی حکومت یا، جیسا کہ معاملہ ہو، صوبائی حکومت کی جانب سے وقتاً فوقتاً ان میں ترمیم کی جا سکتی ہے۔
```
سزا سے ماضی کی طرف تحفظ

(۱) کوئی قانون کسی شخص کو سزا دینے کا اختیار نہیں دے گا۔

ایسے فعل یا ترک فعل کے لیے جو فعل یا ترک فعل کے وقت قانون کے تحت قابلِ سزا نہ تھا۔ یا

کسی جرم کے لیے اس سزا سے زیادہ یا اس سے مختلف نوعیت کی سزا جو جرم کے ارتکاب کے وقت اس جرم کے لیے قانون کے تحت مقرر تھی۔

(۲) شق (۱) یا دفعہ ۲۷۰ میں موجود کوئی بھی بات کسی ایسے قانون پر لاگو نہیں ہوگی جو پاکستان میں کسی بھی وقت چوبیس مارچ، سنہ انیس سو چھپن کے بعد نافذ کسی آئین کی منسوخی یا تخریب کے افعال کو جرم قرار دیتا ہو۔
```
گورنر اپنے عہدے کا حلف اٹھانے سے قبل ہائی کورٹ کے چیف جسٹس کے سامنے تیسری فہرست میں دی گئی صورت کے مطابق حلف لے گا۔
صدر کا حلف

عہدہ سنبھالنے سے قبل صدر پا

,context_eng,context_urdu,predicted_translation_gemini-1.5-pro-002
0,"Existing rules, etc., to continue\n\nUntil the...",۴۲۲۔ ۲۲۔ جب تک کہ متعلقہ مقننہ آ...,موجودہ قوانین وغیرہ کا جاری رہنا۔\n\nآئین کے آ...
1,Protection against retrospective punishment\n\...,کوئی قانون کسی شخص کو مؤثر بہ ماضی سزا سے تحفظ...,```\nسزا سے ماضی کی طرف تحفظ\n\n(۱) کوئی قانون...
2,"Oath of Office\n\nBefore entering upon office,...",ﻋﮩﺪﮮ ﮐﺎ ﺣﻠﻒ\n\nﻋﮩﺪﮮ ﭘﺮ ﻓﺎﺋﺰ ﮨﻮﻧﮯ ﺳﮯ ﻗﺒﻞ، ﮔﻮﺭﻧﺮ...,گورنر اپنے عہدے کا حلف اٹھانے سے قبل ہائی کورٹ...
3,Oath of President\n\nBefore entering upon offi...,صدر کا حلف\n\nعہدہ سنبھالنے سے قبل، صدر پاکستا...,صدر کا حلف\n\nعہدہ سنبھالنے سے قبل صدر پاکستان...
4,National language\n\n(1) The National language...,قومی زبان\n\n(1) پاکستان کی قومی زبان اردو ہے ...,```urdu\nقومی زبان\n\n(۱) پاکستان کی قومی زبان...


------------------------------ Done ------------------------------
Model: gemini-1.5-flash-002


Processing responses:   0%|          | 0/50 [00:00<?, ?item/s]

موجودہ قواعد وغیرہ، جاری رہیں گے

آرٹیکل 240 کے تحت مناسب قانون سازی کرنے تک، شروع ہونے کے دن سے فوراً قبل نافذ تمام قواعد و احکامات، جہاں تک آئین کے تقاضوں کے مطابق ہوں، نافذ رہیں گے اور وفاقی حکومت یا جہاں تک صورتحال ہو صوبائی حکومت وقتاً فوقتاً ان میں ترمیم کر سکتی ہے۔
```urdu
ماضی کی سزا سے تحفظ

(1) کوئی قانون کسی شخص کو سزا دینے کی اجازت نہیں دے گا۔

کسی ایسے عمل یا تقصیر کے لیے جو اس عمل یا تقصیر کے وقت قانون کے تحت قابل سزا نہ تھا؛ یا

کسی جرم کے لیے اس سے زیادہ یا مختلف قسم کی سزا سے جو اس جرم کے ارتکاب کے وقت قانون کے تحت اس جرم کے لیے مقرر تھی۔

(2) شق (1) یا آرٹیکل 270 میں کچھ بھی کسی ایسے قانون پر لاگو نہیں ہوگا جو پاکستان میں 23 مارچ، 1956ء کے بعد کسی بھی وقت نافذ آئین کی منسوخی یا تباہی کے کاموں کو جرم قرار دے۔
```
خدمت کا حلف

عہدے پر فائز ہونے سے قبل، گورنر ہائی کورٹ کے چیف جسٹس کے سامنے تیسری شیڈول میں دیے گئے فارم کے مطابق حلف لے گا۔
صدر کا حلف

عہدے کا چارج سنبھالنے سے قبل، صدر پاکستان کے چیف جسٹس کے سامنے تیسری شیڈول میں دیے گئے فارم کے مطابق حلف لے گا۔
قومی زبان



,context_eng,context_urdu,predicted_translation_gemini-1.5-flash-002
0,"Existing rules, etc., to continue\n\nUntil the...",۴۲۲۔ ۲۲۔ جب تک کہ متعلقہ مقننہ آ...,موجودہ قواعد وغیرہ، جاری رہیں گے\n\nآرٹیکل 240...
1,Protection against retrospective punishment\n\...,کوئی قانون کسی شخص کو مؤثر بہ ماضی سزا سے تحفظ...,```urdu\nماضی کی سزا سے تحفظ\n\n(1) کوئی قانون...
2,"Oath of Office\n\nBefore entering upon office,...",ﻋﮩﺪﮮ ﮐﺎ ﺣﻠﻒ\n\nﻋﮩﺪﮮ ﭘﺮ ﻓﺎﺋﺰ ﮨﻮﻧﮯ ﺳﮯ ﻗﺒﻞ، ﮔﻮﺭﻧﺮ...,خدمت کا حلف\n\nعہدے پر فائز ہونے سے قبل، گورنر...
3,Oath of President\n\nBefore entering upon offi...,صدر کا حلف\n\nعہدہ سنبھالنے سے قبل، صدر پاکستا...,صدر کا حلف\n\nعہدے کا چارج سنبھالنے سے قبل، صد...
4,National language\n\n(1) The National language...,قومی زبان\n\n(1) پاکستان کی قومی زبان اردو ہے ...,قومی زبان\n\n(۱) پاکستان کی قومی زبان اردو ہے،...


------------------------------ Done ------------------------------
